# Wordle helper 

Simple Julia code to help with playing the game Wordle. 

## Loading the code

In [6]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

import DataStructures: counter
import Random: shuffle 

include(joinpath(@__DIR__, "wordle_helper.jl"))

  Activating project at `~/Github/Wordle_helper`


solve_a_game (generic function with 1 method)

## Finding the best starting word

In [7]:
let 
    all_words = get_word_lists()
    find_best_guess(;all_words..., verbose = true)
end 

Progress: 100%|█████████████████████████████████████████| Time: 0:06:34


("roate", 60.42462203023758)

Using only guesses that could be solutions:

In [8]:
let 
    all_words = get_word_lists(; use_all_allowed_guesses = false)
    find_best_guess(;all_words..., verbose = true)
end 

Progress: 100%|█████████████████████████████████████████| Time: 0:01:10


("raise", 61.000431965442765)

Using an entropy measure for the score:

In [9]:
let 
    all_words = get_word_lists()
    find_best_guess(;all_words..., verbose = true, use_entropy = true)
end 

Progress: 100%|█████████████████████████████████████████| Time: 0:06:29


("soare", -5.815104366608849)

In [10]:
let 
    all_words = get_word_lists(; use_all_allowed_guesses = false)
    find_best_guess(;all_words..., verbose = true, use_entropy = true)
end 

Progress: 100%|█████████████████████████████████████████| Time: 0:01:11


("raise", -5.805725970243446)

## Solving some words

Testing some solutions

In [11]:
let 
    all_words = get_word_lists(; use_all_allowed_guesses = false)
    for w in ["epoxy", "lodge", "wound", "awash", "raise"] 
        println(solve(w, "raise"; all_words...))
    end 
end

["raise", "betel", "enjoy", "epoxy"]
["raise", "could", "lodge"]
["raise", "could", "blimp", "awash", "wound"]
["raise", "slash", "about", "awash"]
["raise"]


In [12]:
let 
    all_words = get_word_lists(; use_all_allowed_guesses = false)
    for w in ["epoxy", "lodge", "wound", "awash", "raise"] 
        println(solve(w, "raise"; all_words..., use_entropy = true))
    end 
end

["raise", "betel", "envoy", "epoxy"]
["raise", "could", "lodge"]
["raise", "mulch", "bound", "awful", "wound"]
["raise", "slash", "about", "awash"]
["raise"]


### Hard mode

In [13]:
let 
    all_words = get_word_lists(; use_all_allowed_guesses = false)
    for w in ["epoxy", "lodge", "wound", "awash", "raise"] 
        println(solve(w, "roate"; all_words..., hard_mode = true))
    end
end  

["roate", "demon", "epoch", "epoxy"]
["roate", "louse", "lodge"]
["roate", "godly", "bound", "found", "hound", "mound", "pound", "sound", "wound"]
["roate", "slack", "awash"]
["roate", "raise"]


## Solving all games

In [14]:
function solve_all_words(; use_all_allowed_guesses = true, hard_mode = false, first_guess = "raise", use_entropy = false)
    all_words = get_word_lists(; use_all_allowed_guesses)
    p = Progress(length(all_words.words))
    return ThreadsX.map(shuffle(all_words.words)) do word 
        next!(p)
        solve(word, first_guess; all_words..., hard_mode, use_entropy)
    end
end

solve_all_words (generic function with 1 method)

In [15]:
let 
    sol = solve_all_words()
    counter(map(x -> length(x), sol)) |> collect |> sort
end 

Progress: 100%|█████████████████████████████████████████| Time: 0:16:02


5-element Vector{Pair{Int64, Int64}}:
 1 => 1
 2 => 57
 3 => 1117
 4 => 1078
 5 => 62

This algorithm breaks all Wordle puzzles under 5 guesses. 

Using the entropy score:

In [16]:
let 
    sol = solve_all_words(use_entropy = true)
    counter(map(x -> length(x), sol)) |> collect |> sort
end 

Progress: 100%|█████████████████████████████████████████| Time: 0:16:01


6-element Vector{Pair{Int64, Int64}}:
 1 => 1
 2 => 58
 3 => 1161
 4 => 1032
 5 => 61
 6 => 2

Using only guesses that are potential solutions: 

In [17]:
let 
    sol = solve_all_words(; use_all_allowed_guesses = false, first_guess = "raise")
    counter(map(x -> length(x), sol)) |> collect |> sort
end

Progress: 100%|█████████████████████████████████████████| Time: 0:02:53


5-element Vector{Pair{Int64, Int64}}:
 1 => 1
 2 => 61
 3 => 1073
 4 => 1090
 5 => 90

### Hard mode

In [18]:
let 
    sol = solve_all_words(; hard_mode = true)
    counter(map(x -> length(x), sol)) |> collect |> sort
end 

Progress: 100%|█████████████████████████████████████████| Time: 0:00:10


8-element Vector{Pair{Int64, Int64}}:
 1 => 1
 2 => 131
 3 => 957
 4 => 945
 5 => 225
 6 => 42
 7 => 11
 8 => 3

Using the entropy score:

In [19]:
let 
    sol = solve_all_words(; hard_mode = true, use_entropy = true)
    counter(map(x -> length(x), sol)) |> collect |> sort
end 

Progress: 100%|█████████████████████████████████████████| Time: 0:00:10


8-element Vector{Pair{Int64, Int64}}:
 1 => 1
 2 => 131
 3 => 995
 4 => 917
 5 => 212
 6 => 46
 7 => 11
 8 => 2

Using only guesses that are potential solutions:

In [20]:
let 
    sol = solve_all_words(; hard_mode = true, use_all_allowed_guesses = false, first_guess = "raise")
    counter(map(x -> length(x), sol)) |> collect |> sort
end 

Progress: 100%|█████████████████████████████████████████| Time: 0:00:09


8-element Vector{Pair{Int64, Int64}}:
 1 => 1
 2 => 131
 3 => 957
 4 => 945
 5 => 225
 6 => 42
 7 => 11
 8 => 3

## Breaking one Wordle

### Wordle 220

In [21]:
all_words = get_word_lists()
game_words = get_word_lists().words

new_guess_and_update!(game_words, "raise", [1, 1, 0, 1, 0], all_words.words)

("chant", 2.3333333333333335)

In [22]:
new_guess_and_update!(game_words, "chant", [0, 0, 1, 0, 0], all_words.words)

("solar", 0.6666666666666666)

In [23]:
new_guess_and_update!(game_words, "solar", [2, 0, 0, 2, 2], all_words.words)

("sugar", 0.0)

### Wordle 223 

In [24]:
all_words = get_word_lists()
game_words = get_word_lists().words

new_guess_and_update!(game_words, "raise", [1, 0, 0, 0, 1], all_words.words)

("outer", 8.676470588235293)

In [25]:
new_guess_and_update!(game_words, "outer", [0, 0, 0, 1, 1], all_words.words)

("clerk", 1.7857142857142858)

In [26]:
new_guess_and_update!(game_words, "clerk", [0, 0, 1, 1, 1], all_words.words)

("jerky", 0.5)

In [27]:
new_guess_and_update!(game_words, "jerky", [0, 2, 2, 2, 2], all_words.words)

("perky", 0.0)

### Hard mode. 


Wordle 222 

In [28]:
all_words = get_word_lists()
game_words = get_word_lists().words

new_guess_and_update!(game_words, "raise", [0, 0, 0, 0, 0], game_words, hard_mode = true)

("could", 6.267857142857143)

In [29]:
new_guess_and_update!(game_words, "could", [0, 2, 2, 0, 0], game_words, hard_mode = true)

("mouth", 0.8571428571428571)

In [30]:
new_guess_and_update!(game_words, "mouth", [2, 2, 2, 1, 0], game_words, hard_mode = true)

("mount", 0.0)